In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.colab import drive
drive.mount('/content/drive')
print("Files in Glove-in-bangla:")
!ls /content/drive/My\ Drive/Research/Vector/Glove-in-bangla

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Files in Glove-in-bangla:
corpus.txt	       preprocessing.py  Siyamrupali.ttf
Glove-in-bangla.ipynb  __pycache__	 text3.txt
log		       saved_model	 tf_glove.py


In [0]:
with open('/content/drive/My Drive/Research/Vector/Glove-in-bangla/corpus.txt') as f:
    singleLine = f.readline()
    listoflist = []
    words = []
    while singleLine:
        words = singleLine.split()
        listoflist.append(words)
        singleLine = f.readline()

In [5]:
%cd /content/drive/My\ Drive/Research/Vector/Glove-in-bangla
!ls
import tf_glove

/content/drive/My Drive/Research/Vector/Glove-in-bangla
corpus.txt	       preprocessing.py  Siyamrupali.ttf
Glove-in-bangla.ipynb  __pycache__	 text3.txt
log		       saved_model	 tf_glove.py


# Instantiating the model

To create a new GloVe model, simply call `tf_glove.GloVeModel()`:

In [0]:
model = tf_glove.GloVeModel(embedding_size=300, context_size=10, min_occurrences=1000,
                            learning_rate=0.05, batch_size=512)

`GloVeModel()` has several parameters:
- **`embedding_size`**: the target dimensionality of the trained word representations. Typically between 50 and 300.
- **`context_size`**: how many tokens on either side of a given word to include in each context window. Can be either a tuple of two ints, indicating how many token on the left and right to include, or a single int, which will be interpreted to mean symmetric context.
- **`max_vocab_size`** *(Optional)*: the maximum size of the model's vocabulary. The model's vocabulary will be the most frequently occurring words in the corpus up to this amount. The default is 100,000.
- **`min_occurrences`** *(Optional)*: the minimum number of times a word must have appeared in the corpus to be included in the model's vocabulary. Default is 1.
- **`scaling_factor`** *(Optional)*: the alpha term in Eqn. 9 of Pennington et al.'s paper. Default is 3/4, which is the paper's recommendation
- **`cooccurrence_cap`** *(Optional)*: the x_max term in Eqn. 9 of Pennington et al.'s paper. Default is 100, which is the paper's recommendation
- **`batch_size`** *(Optional)*: the number of cooccurrences per minibatch of in training. Default is 512, which seems to work well on my machine. If training is very slow, consider playing with this.
- **`learning_rate`** *(Optional)*: the Adagrad learning rate used in training. Default is 0.05, which is the paper's recommendation

# Reading the corpus

tf_glove needs to be fit to a corpus in order to learn word representations. To do this, we'll use `GloVeModel.fit_to_corpus(corpus)`. This method expects an iterable of iterables of strings, where each string is a token, like this:

`[["this", "is", "a", "comment", "."], ["this", "is", "another", "comment", "."]]`

That was a list of lists, but any iterable of iterables of strings should work.

### Note on getting the dataset (if you want to follow along with these examples exactly)

For these examples, I'm going to use the dataset of Reddit comments described here: https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment

tf_glove is designed to work with any corpus, so there's no need to download this dataset. However, if you'd like to, that post has a link to a torrent for all of the comments as well as a link for just the comments from January 2015. Even just the January 2015 file is quite large (~5 GB).

I downloaded it and used

`$ head -n 1000000 RC_2015-01 > /path/to/RC_2015-01-1m_sample`

to get the 1 million comment sample file referenced below. You could also use 100k if you want to save some time. 1 million comments takes ~15 minutes to fit on my machine.

## The code:

In [0]:
# import re
# import nltk

# def extract_reddit_comments(path):
#     # A regex for extracting the comment body from one line of JSON (faster than parsing)
#     body_snatcher = re.compile(r"\{.*?(?<!\\)\"body(?<!\\)\":(?<!\\)\"(.*?)(?<!\\)\".*}")
#     with open(path) as file_:
#         for line in file_:
#             match = body_snatcher.match(line)
#             if match:
#                 body = match.group(1)
#                 # Ignore deleted comments
#                 if not body == '[deleted]':
#                     # Return the comment as a string (not yet tokenized)
#                     yield body
                        
# def tokenize_comment(comment_str):
#     # Use the excellent NLTK to tokenize the comment body
#     #
#     # Note that we're lower-casing the comments here. tf_glove is case-sensitive,
#     # so if you want 'You' and 'you' to be considered the same word, be sure to lower-case everything.
#     return nltk.wordpunct_tokenize(comment_str.lower())

# def reddit_comment_corpus(path):
#     # A generator that returns lists of tokens representing individual words in the comment
#     return (tokenize_comment(comment) for comment in extract_reddit_comments(path))

# # Replace the path with the path to your corpus file
# corpus = reddit_comment_corpus("/media/grady/PrimeMover/Datasets/RC_2015-01-1m_sample")

Now, to fit the model to the corpus:

In [7]:
model.fit_to_corpus(listoflist)

W0624 10:10:50.500548 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:71: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0624 10:10:50.511616 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:79: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 10:10:50.560105 140019811432320 deprecation.py:323] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:98: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0624 10:10:50.572716 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:103: The name tf.log is deprecated. Please use tf.math.log instead.

W0624 10:10:50.573966 14001981

# Training the model

GloVeModel.fit_to_corpus() builds the vocabulary and cooccurrence matrix that will be used in training, but it doesn't actually train the word representations. It's time to kick off TensorFlow and train the model for real:

In [0]:
model.train(num_epochs=100, checkpoint_dir="/content/drive/My Drive/Research/Vector/Glove-in-bangla/saved_model", log_dir="log/example", summary_batch_interval=1000)

W0624 10:12:15.878669 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:129: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0624 10:12:17.063442 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:132: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Writing TensorBoard summaries to log/example


W0624 10:12:17.327284 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:133: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0624 10:12:17.426094 140019811432320 deprecation_wrapper.py:119] From /content/drive/My Drive/Research/Vector/Glove-in-bangla/tf_glove.py:134: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0624 10:12:17.904421 140019811432320 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Restoring previous checkpoint: /content/drive/My Drive/Research/Vector/Glove-in-bangla/saved_model/model.ckpt-134600
 Epoch 0: Model is saved Elapsed: 00:00:26.82 Steps: 1346 Loss: 15.118061065673828 

 Epoch 1: Model is saved Elapsed: 00:00:54.33 Steps: 2692 Loss: 16.73320770263672 

 Epoch 2: Model is saved Elapsed: 00:01:26.63 Steps: 4038 Loss: 17.203197479248047 



W0624 10:14:19.213001 140019811432320 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


 Epoch 3: Model is saved Elapsed: 00:02:03.38 Steps: 5384 Loss: 15.082965850830078 

 Epoch 4: Model is saved Elapsed: 00:02:39.57 Steps: 6730 Loss: 16.335718154907227 

 Epoch 5: Model is saved Elapsed: 00:03:15.77 Steps: 8076 Loss: 15.220139503479004 

 Epoch 6: Model is saved Elapsed: 00:03:52.35 Steps: 9422 Loss: 14.798406600952148 

 Epoch 7: Model is saved Elapsed: 00:04:28.88 Steps: 10768 Loss: 17.35226821899414 

 Epoch 8: Model is saved Elapsed: 00:05:05.24 Steps: 12114 Loss: 14.784404754638672 

 Epoch 9: Model is saved Elapsed: 00:05:41.99 Steps: 13460 Loss: 13.656848907470703 

 Epoch 10: Model is saved Elapsed: 00:06:18.79 Steps: 14806 Loss: 14.132999420166016 

 Epoch 11: Model is saved Elapsed: 00:06:56.89 Steps: 16152 Loss: 14.438039779663086 

 Epoch 12: Model is saved Elapsed: 00:07:38.25 Steps: 17498 Loss: 14.068052291870117 

 Epoch 13: Model is saved Elapsed: 00:08:18.94 Steps: 18844 Loss: 15.917762756347656 

 Epoch 14: Model is saved Elapsed: 00:09:00.18 Steps: 2

`GloVeModel.train()` has a few parameters:
- **`num_epochs`**: How many passes through the cooccurrence matrix the training should make. The paper recommends at least 50 for `embedding_size` < 300, and 100 otherwise.
- **`log_dir`** *(Optional)*: The path of the directory in which to log summaries for TensorBoard and t-SNE visualizations. Default is `None`, i.e. don't log anything.
- **`summary_batch_interval`** *(Optional)*: How many minibatches between logging events for TensorBoard. Default is 1000.
- **`tsne_epoch_interval`** *(Optional)*: How many epochs (full passes through cooccurrence matrix) between outputting a t-SNE visualization of the model's embeddings for the most frequent 1000 words in the vocabulary. Default is None, i.e. don't output t-SNE visualizations during training.

# Checking out the results

Now that we've trained the model, let's look at the results.

Use `GloVeModel.embedding_for()` to get the trained embedding for a single word:

In [0]:
model.embedding_for("বাংলাদেশের")

array([-0.7167677 , -0.38585076, -1.2564719 ,  0.6048701 ,  0.9983499 ,
       -0.6561352 ,  0.58636254,  0.250298  , -0.20085195,  0.88739324,
       -0.06227261, -0.16828527,  0.34972733, -0.21013565, -0.05010937,
       -0.49934825,  0.5918076 , -0.423181  , -0.79169136, -0.7910562 ,
       -0.6646234 , -0.00484452, -0.21082681, -0.24385452,  0.7502422 ,
       -1.1131271 ,  0.6395327 ,  0.5253399 ,  0.2103363 ,  0.49985048,
       -0.35401842,  0.08719155, -1.5429426 , -0.44470173,  0.12148225,
        0.09441686,  0.7596359 ,  0.32775313, -0.30396938,  0.02917969,
        1.000686  , -0.6873554 , -0.2691379 , -1.0054915 , -0.80284387,
        0.3039051 ,  0.38538867, -0.5976006 , -0.26354825, -0.1503103 ,
       -0.13959584,  0.00162068,  0.30284977, -0.2787998 ,  0.20144096,
        0.02304388,  0.11521374,  1.5319884 , -0.25872156, -0.5645089 ,
        0.04073019, -0.5498246 ,  0.59661627, -0.24174398,  0.41592818,
        0.45187503, -0.1710605 , -0.37482455, -0.77000535,  0.61

You can also get the model's embeddings for every word in the vocabulary like this:

In [0]:
model.embeddings

array([[ 2.4801731e-02, -6.8627572e-01, -4.4789314e-02, ...,
        -8.2306689e-01,  7.7048725e-01,  8.9217269e-01],
       [-4.3665755e-01,  1.2864321e-03, -2.9219371e-01, ...,
        -7.1725792e-01,  1.6258982e-01, -7.9128635e-01],
       [-4.6904767e-01,  4.1739598e-01, -2.3801640e-01, ...,
         1.3364501e+00, -4.8624003e-01, -3.8267452e-01],
       ...,
       [-1.0566947e+00, -6.6683561e-01,  3.6931178e-01, ...,
         4.0226436e-01,  5.5630445e-01, -3.5801595e-01],
       [-7.8112650e-01,  1.0747304e+00,  6.2679887e-02, ...,
        -2.7119958e-01, -5.7600695e-01, -8.0436006e-02],
       [-1.0481548e-01,  4.6539104e-01, -5.5051619e-01, ...,
        -4.0089184e-01, -8.5911021e-02,  5.3729117e-03]], dtype=float32)

`GloVeModel.embeddings` will give you a NumPy matrix where each row is the model's embedding for a single word.

To make use of this, you'll want to know what row corresponds to a particular word. You can do that with `GloVeModel.id_for_word`:

In [0]:
model.embeddings[model.id_for_word('বাংলাদেশের')]

array([-0.7167677 , -0.38585076, -1.2564719 ,  0.6048701 ,  0.9983499 ,
       -0.6561352 ,  0.58636254,  0.250298  , -0.20085195,  0.88739324,
       -0.06227261, -0.16828527,  0.34972733, -0.21013565, -0.05010937,
       -0.49934825,  0.5918076 , -0.423181  , -0.79169136, -0.7910562 ,
       -0.6646234 , -0.00484452, -0.21082681, -0.24385452,  0.7502422 ,
       -1.1131271 ,  0.6395327 ,  0.5253399 ,  0.2103363 ,  0.49985048,
       -0.35401842,  0.08719155, -1.5429426 , -0.44470173,  0.12148225,
        0.09441686,  0.7596359 ,  0.32775313, -0.30396938,  0.02917969,
        1.000686  , -0.6873554 , -0.2691379 , -1.0054915 , -0.80284387,
        0.3039051 ,  0.38538867, -0.5976006 , -0.26354825, -0.1503103 ,
       -0.13959584,  0.00162068,  0.30284977, -0.2787998 ,  0.20144096,
        0.02304388,  0.11521374,  1.5319884 , -0.25872156, -0.5645089 ,
        0.04073019, -0.5498246 ,  0.59661627, -0.24174398,  0.41592818,
        0.45187503, -0.1710605 , -0.37482455, -0.77000535,  0.61

And if you want to see a 2D visualization of the learned vector space, you can use `GloVeModel.generate_tsne()`:

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib
import matplotlib.font_manager
# !wget https://www.omicronlab.com/download/fonts/Siyamrupali.ttf
!cp Siyamrupali.ttf /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
!cp Siyamrupali.ttf /usr/share/fonts/truetype/

matplotlib.font_manager._rebuild()
matplotlib.rc('font', family='Siyam Rupali')
model.generate_tsne()

You might want to open that image in a new tab.

With no parameters, `GloVeModel.generate_tsne()` can be used interactively like in this notebook, but it also has parameters that will let you save the visualization to a file and adjust the size of the image and how many words appear:

- **`path`** *(Optional)*: The path at which to save the generated PNG image. Default is None, which only really makes sense for interactive environments.
- **`size`** *(Optional)*: A tuple of (width, height) in *inches*. (Yeah, I know right? This is inherited from matplotlib.) Default is 100 x 100.  
- **`word_count`** *(Optional)*: How many words to plot in the visualization. Default is 1000, which works fairly well for a (100 x 100) visualization.
